## Machine Learning project
[MI-net, instance space](#section_id)
[MI-net deep supervision, instance space](#MInetdeepsuper)


In [3]:
pip install -r requirements.txt

  Cloning https://github.com/chlorochrule/cknn to /private/var/folders/dc/1bymglpd6198p4cqkyfcnrqw0000gn/T/pip-req-build-g_yrvuo2
  Running command git clone --filter=blob:none --quiet https://github.com/chlorochrule/cknn /private/var/folders/dc/1bymglpd6198p4cqkyfcnrqw0000gn/T/pip-req-build-g_yrvuo2
  Resolved https://github.com/chlorochrule/cknn to commit 7d05c5049da72a573bd486fca6647f8b0376243c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/garydoranjr/misvm.git to /private/var/folders/dc/1bymglpd6198p4cqkyfcnrqw0000gn/T/pip-install-kjjbln05/misvm_73298e0f190f474abba4697f191afab0
  Running command git clone --filter=blob:none --quiet https://github.com/garydoranjr/misvm.git /private/var/folders/dc/1bymglpd6198p4cqkyfcnrqw0000gn/T/pip-install-kjjbln05/misvm_73298e0f190f474abba4697f191afab0
  Resolved https://github.com/garydoranjr/misvm.git to commit b2118fe04d98c00436

  Created wheel for misvm: filename=misvm-1.0-py3-none-any.whl size=27219 sha256=3c1325fac11446d6676afa75fe68c115707661071fd3c52cd6176ec053c103bd
  Stored in directory: /private/var/folders/dc/1bymglpd6198p4cqkyfcnrqw0000gn/T/pip-ephem-wheel-cache-kc2uslay/wheels/7b/c8/cf/eef87750d4a940c372521f601ce2cdf30ccfe2d3259f4a9ddb
Successfully built misvm
Note: you may need to restart the kernel to use updated packages.


### Embedded-Space


### MI-net
<a id='section_id'></a>

In [4]:
import numpy as np
import sys
import time
import random
from random import shuffle
import argparse

from keras.models import Model
from keras.optimizers import SGD
from keras.regularizers import l2
from keras.layers import Input, Dense, Layer, Dropout

from mil_nets.dataset import load_dataset
from mil_nets.layer import Feature_pooling
from mil_nets.metrics import bag_accuracy
from mil_nets.objectives import bag_loss
from mil_nets.utils import convertToBatch

In [5]:
def test_eval(model, test_set):
    """Evaluate on testing set.
    Parameters
    -----------------
    model : keras.engine.training.Model object
        The training MI-Net model.
    test_set : list
        A list of testing set contains all training bags features and labels.
    Returns
    -----------------
    test_loss : float
        Mean loss of evaluating on testing set.
    test_acc : float
        Mean accuracy of evaluating on testing set.
    """
    num_test_batch = len(test_set)
    test_loss = np.zeros((num_test_batch, 1), dtype=np.float32)
    test_acc = np.zeros((num_test_batch, 1), dtype=np.float32)
    for ibatch, batch in enumerate(test_set):
        result = model.test_on_batch({'input':batch[0].astype(np.float32)}, {'fp':batch[1].astype(np.float32)})
        test_loss[ibatch] = result[0]
        test_acc[ibatch][0] = result[1]
    return np.mean(test_loss), np.mean(test_acc)

def train_eval(model, train_set):
    """Evaluate on training set.
    Parameters
    -----------------
    model : keras.engine.training.Model object
        The training MI-Net model.
    train_set : list
        A list of training set contains all training bags features and labels.
    Returns
    -----------------
    test_loss : float
        Mean loss of evaluating on traing set..astype(np.float32)
    test_acc : float
        Mean accuracy of evaluating on testing set.
    """
    num_train_batch = len(train_set)
    train_loss = np.zeros((num_train_batch, 1), dtype=np.float32)
    train_acc = np.zeros((num_train_batch, 1), dtype=np.float32)
    shuffle(train_set)
    for ibatch, batch in enumerate(train_set):
        result = model.train_on_batch({'input':batch[0].astype(np.float32)}, {'fp':batch[1].astype(np.float32)})
        train_loss[ibatch] = result[0]
        train_acc[ibatch][0] = result[1]
    return np.mean(train_loss), np.mean(train_acc)

def MI_Net(dataset):
    """Train and evaluate on MI-Net.
    Parameters
    -----------------
    dataset : dict
        A dictionary contains all dataset information. We split train/test by keys.
    Returns
    -----------------
    test_acc : float
        Testing accuracy of MI-Net.
    """
    weight_decay=0.005
    init_lr=5e-4
    pooling_mode='max'
    momentum=0.9
    max_epoch=50
    # load data and convert type
    train_bags = dataset['train']
    test_bags = dataset['test']

    # convert bag to batch
    train_set = convertToBatch(train_bags)
    test_set = convertToBatch(test_bags)
    dimension = train_set[0][0].shape[1]

    # data: instance feature, n*d, n = number of training instance
    data_input = Input(shape=(dimension,), dtype='float32', name='input')

    # fully-connected
    fc1 = Dense(256, activation='relu', kernel_regularizer=l2(weight_decay))(data_input)
    fc2 = Dense(128, activation='relu', kernel_regularizer=l2(weight_decay))(fc1)
    fc3 = Dense(64, activation='relu', kernel_regularizer=l2(weight_decay))(fc2)

    # dropout
    dropout = Dropout(rate=0.5)(fc3)

    # features pooling
    fp = Feature_pooling(output_dim=1, kernel_regularizer=l2(weight_decay), pooling_mode=pooling_mode, name='fp')(dropout)

    model = Model(inputs=[data_input], outputs=[fp])
    sgd = SGD(lr=init_lr, decay=1e-4, momentum=momentum, nesterov=True)
    model.compile(loss=bag_loss, optimizer=sgd, metrics=[bag_accuracy])

    # train model
    t1 = time.time()
    num_batch = len(train_set)
    for epoch in range(max_epoch):
        train_loss, train_acc = train_eval(model, train_set)
        test_loss, test_acc = test_eval(model, test_set)
        print('epoch=', epoch, '  train_loss= {:.3f}'.format(train_loss), '  train_acc= {:.3f}'.format(train_acc), '  test_loss={:.3f}'.format(test_loss), '  test_acc= {:.3f}'.format(test_acc))
    t2 = time.time()
    print('run time:', (t2-t1) / 60, 'min')
    print('test_acc={:.3f}'.format(test_acc))

    return test_acc

In [6]:
# perform five times 10-fold cross-validation experiments
run = 5
n_folds = 10
acc = np.zeros((run, n_folds), dtype=np.float32)
for irun in range(run):
    dataset = load_dataset('musk1', n_folds)
    for ifold in range(n_folds):
        print('run=', irun, '  fold=', ifold)
        acc[irun][ifold] = MI_Net(dataset[ifold])
print('MI-Net mean accuracy = ', np.mean(acc))
print('std = ', np.std(acc))

run= 0   fold= 0


/Users/tibovanheule/Desktop/MachineLearning/venv/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


epoch= 0   train_loss= 2.929   train_acc= 0.561   test_loss=2.835   test_acc= 0.600
epoch= 1   train_loss= 2.597   train_acc= 0.890   test_loss=2.651   test_acc= 0.900
epoch= 2   train_loss= 2.509   train_acc= 0.915   test_loss=2.577   test_acc= 0.900
epoch= 3   train_loss= 2.443   train_acc= 0.902   test_loss=2.628   test_acc= 0.900
epoch= 4   train_loss= 2.365   train_acc= 0.963   test_loss=2.574   test_acc= 0.900
epoch= 5   train_loss= 2.314   train_acc= 0.976   test_loss=2.601   test_acc= 0.900
epoch= 6   train_loss= 2.268   train_acc= 0.976   test_loss=2.571   test_acc= 0.900
epoch= 7   train_loss= 2.245   train_acc= 1.000   test_loss=2.566   test_acc= 0.900
epoch= 8   train_loss= 2.203   train_acc= 0.988   test_loss=2.573   test_acc= 0.900
epoch= 9   train_loss= 2.206   train_acc= 0.976   test_loss=2.556   test_acc= 0.900
epoch= 10   train_loss= 2.172   train_acc= 1.000   test_loss=2.493   test_acc= 0.900
epoch= 11   train_loss= 2.145   train_acc= 1.000   test_loss=2.534   test_a

epoch= 46   train_loss= 1.723   train_acc= 0.988   test_loss=1.835   test_acc= 1.000
epoch= 47   train_loss= 1.701   train_acc= 1.000   test_loss=1.815   test_acc= 1.000
epoch= 48   train_loss= 1.687   train_acc= 1.000   test_loss=1.801   test_acc= 1.000
epoch= 49   train_loss= 1.679   train_acc= 1.000   test_loss=1.793   test_acc= 1.000
run time: 0.808122165997823 min
test_acc=1.000
run= 0   fold= 2
epoch= 0   train_loss= 3.009   train_acc= 0.663   test_loss=2.803   test_acc= 0.667
epoch= 1   train_loss= 2.658   train_acc= 0.819   test_loss=2.785   test_acc= 0.778
epoch= 2   train_loss= 2.498   train_acc= 0.940   test_loss=2.776   test_acc= 0.778
epoch= 3   train_loss= 2.424   train_acc= 0.952   test_loss=2.659   test_acc= 0.889
epoch= 4   train_loss= 2.350   train_acc= 0.988   test_loss=2.679   test_acc= 0.889
epoch= 5   train_loss= 2.308   train_acc= 0.976   test_loss=2.449   test_acc= 0.889
epoch= 6   train_loss= 2.274   train_acc= 1.000   test_loss=2.434   test_acc= 0.889
epoch= 7

epoch= 42   train_loss= 1.744   train_acc= 1.000   test_loss=1.776   test_acc= 1.000
epoch= 43   train_loss= 1.734   train_acc= 1.000   test_loss=1.765   test_acc= 1.000
epoch= 44   train_loss= 1.724   train_acc= 1.000   test_loss=1.750   test_acc= 1.000
epoch= 45   train_loss= 1.717   train_acc= 1.000   test_loss=1.740   test_acc= 1.000
epoch= 46   train_loss= 1.707   train_acc= 1.000   test_loss=1.740   test_acc= 1.000
epoch= 47   train_loss= 1.694   train_acc= 1.000   test_loss=1.727   test_acc= 1.000
epoch= 48   train_loss= 1.684   train_acc= 1.000   test_loss=1.714   test_acc= 1.000
epoch= 49   train_loss= 1.675   train_acc= 1.000   test_loss=1.705   test_acc= 1.000
run time: 0.6962620496749878 min
test_acc=1.000
run= 0   fold= 4
epoch= 0   train_loss= 2.999   train_acc= 0.578   test_loss=2.826   test_acc= 0.667
epoch= 1   train_loss= 2.614   train_acc= 0.867   test_loss=2.793   test_acc= 0.778
epoch= 2   train_loss= 2.476   train_acc= 0.940   test_loss=2.821   test_acc= 0.556
epo

...

epoch= 28   train_loss= 1.905   train_acc= 1.000   test_loss=2.181   test_acc= 0.900
epoch= 29   train_loss= 1.891   train_acc= 1.000   test_loss=2.137   test_acc= 0.900
epoch= 30   train_loss= 1.875   train_acc= 1.000   test_loss=2.125   test_acc= 0.900
epoch= 31   train_loss= 1.866   train_acc= 1.000   test_loss=2.111   test_acc= 0.900
epoch= 32   train_loss= 1.856   train_acc= 1.000   test_loss=2.096   test_acc= 0.900
epoch= 33   train_loss= 1.842   train_acc= 1.000   test_loss=2.093   test_acc= 0.900
epoch= 34   train_loss= 1.831   train_acc= 1.000   test_loss=2.092   test_acc= 0.900
epoch= 35   train_loss= 1.821   train_acc= 1.000   test_loss=2.053   test_acc= 0.900
epoch= 36   train_loss= 1.807   train_acc= 1.000   test_loss=2.045   test_acc= 0.900
epoch= 37   train_loss= 1.796   train_acc= 1.000   test_loss=2.048   test_acc= 0.900
epoch= 38   train_loss= 1.794   train_acc= 1.000   test_loss=2.038   test_acc= 0.900
epoch= 39   train_loss= 1.773   train_acc= 1.000   test_loss=2.03

epoch= 24   train_loss= 1.952   train_acc= 1.000   test_loss=1.990   test_acc= 1.000
epoch= 25   train_loss= 1.940   train_acc= 1.000   test_loss=1.977   test_acc= 1.000
epoch= 26   train_loss= 1.922   train_acc= 1.000   test_loss=1.965   test_acc= 1.000
epoch= 27   train_loss= 1.912   train_acc= 1.000   test_loss=1.953   test_acc= 1.000
epoch= 28   train_loss= 1.901   train_acc= 1.000   test_loss=1.946   test_acc= 1.000
epoch= 29   train_loss= 1.889   train_acc= 1.000   test_loss=1.930   test_acc= 1.000
epoch= 30   train_loss= 1.870   train_acc= 1.000   test_loss=1.915   test_acc= 1.000
epoch= 31   train_loss= 1.861   train_acc= 1.000   test_loss=1.901   test_acc= 1.000
epoch= 32   train_loss= 1.850   train_acc= 1.000   test_loss=1.889   test_acc= 1.000
epoch= 33   train_loss= 1.837   train_acc= 1.000   test_loss=1.878   test_acc= 1.000
epoch= 34   train_loss= 1.829   train_acc= 1.000   test_loss=1.869   test_acc= 1.000
epoch= 35   train_loss= 1.813   train_acc= 1.000   test_loss=1.85

epoch= 20   train_loss= 2.014   train_acc= 1.000   test_loss=2.243   test_acc= 0.889
epoch= 21   train_loss= 1.999   train_acc= 1.000   test_loss=2.260   test_acc= 0.889
epoch= 22   train_loss= 1.987   train_acc= 1.000   test_loss=2.223   test_acc= 0.889
epoch= 23   train_loss= 1.970   train_acc= 1.000   test_loss=2.229   test_acc= 0.889
epoch= 24   train_loss= 1.957   train_acc= 1.000   test_loss=2.190   test_acc= 0.889
epoch= 25   train_loss= 1.945   train_acc= 1.000   test_loss=2.187   test_acc= 0.889
epoch= 26   train_loss= 1.930   train_acc= 1.000   test_loss=2.171   test_acc= 0.889
epoch= 27   train_loss= 1.917   train_acc= 1.000   test_loss=2.162   test_acc= 0.889
epoch= 28   train_loss= 1.907   train_acc= 1.000   test_loss=2.141   test_acc= 0.889
epoch= 29   train_loss= 1.894   train_acc= 1.000   test_loss=2.142   test_acc= 0.889
epoch= 30   train_loss= 1.878   train_acc= 1.000   test_loss=2.119   test_acc= 0.889
epoch= 31   train_loss= 1.873   train_acc= 1.000   test_loss=2.11

epoch= 16   train_loss= 2.076   train_acc= 1.000   test_loss=2.167   test_acc= 1.000
epoch= 17   train_loss= 2.069   train_acc= 1.000   test_loss=2.149   test_acc= 1.000
epoch= 18   train_loss= 2.040   train_acc= 1.000   test_loss=2.136   test_acc= 1.000
epoch= 19   train_loss= 2.025   train_acc= 1.000   test_loss=2.119   test_acc= 1.000
epoch= 20   train_loss= 2.011   train_acc= 1.000   test_loss=2.118   test_acc= 1.000
epoch= 21   train_loss= 1.999   train_acc= 1.000   test_loss=2.086   test_acc= 1.000
epoch= 22   train_loss= 1.985   train_acc= 1.000   test_loss=2.079   test_acc= 1.000
epoch= 23   train_loss= 1.970   train_acc= 1.000   test_loss=2.053   test_acc= 1.000
epoch= 24   train_loss= 1.954   train_acc= 1.000   test_loss=2.052   test_acc= 1.000
epoch= 25   train_loss= 1.946   train_acc= 1.000   test_loss=2.045   test_acc= 1.000
epoch= 26   train_loss= 1.936   train_acc= 1.000   test_loss=2.036   test_acc= 1.000
epoch= 27   train_loss= 1.918   train_acc= 1.000   test_loss=2.01

epoch= 12   train_loss= 2.138   train_acc= 1.000   test_loss=2.207   test_acc= 1.000
epoch= 13   train_loss= 2.116   train_acc= 1.000   test_loss=2.198   test_acc= 1.000
epoch= 14   train_loss= 2.095   train_acc= 1.000   test_loss=2.172   test_acc= 1.000
epoch= 15   train_loss= 2.100   train_acc= 0.988   test_loss=2.177   test_acc= 1.000
epoch= 16   train_loss= 2.072   train_acc= 1.000   test_loss=2.148   test_acc= 1.000
epoch= 17   train_loss= 2.045   train_acc= 1.000   test_loss=2.129   test_acc= 1.000
epoch= 18   train_loss= 2.032   train_acc= 1.000   test_loss=2.119   test_acc= 1.000
epoch= 19   train_loss= 2.020   train_acc= 1.000   test_loss=2.090   test_acc= 1.000
epoch= 20   train_loss= 2.007   train_acc= 1.000   test_loss=2.089   test_acc= 1.000
epoch= 21   train_loss= 1.992   train_acc= 1.000   test_loss=2.058   test_acc= 1.000
epoch= 22   train_loss= 1.976   train_acc= 1.000   test_loss=2.054   test_acc= 1.000
epoch= 23   train_loss= 1.963   train_acc= 1.000   test_loss=2.03

### MI-net pooling deep supervision

In [7]:
import numpy as np
import sys
import time
import random
from random import shuffle
import argparse

from keras.models import Model
from keras.optimizers import SGD
from keras.regularizers import l2
from keras.layers import Input, Dense, Layer, Dropout, average

from mil_nets.dataset import load_dataset
from mil_nets.layer import Feature_pooling
from mil_nets.metrics import bag_accuracy
from mil_nets.objectives import bag_loss
from mil_nets.utils import convertToBatch

In [9]:
def test_eval(model, test_set):
    """Evaluate on testing set.
    Parameters
    -----------------
    model : keras.engine.training.Model object
        The training MI-Net with deep supervision model.
    test_set : list
        A list of testing set contains all training bags features and labels.
    Returns
    -----------------
    test_loss : float
        Mean loss of evaluating on testing set.
    test_acc : float
        Mean accuracy of evaluating on testing set.
    """
    num_test_batch = len(test_set)
    test_loss = np.zeros((num_test_batch, 1), dtype=np.float32)
    test_acc = np.zeros((num_test_batch, 1), dtype=np.float32)
    for ibatch, batch in enumerate(test_set):
        result = model.test_on_batch({'input':batch[0].astype(np.float32)}, {'fp1':batch[1].astype(np.float32), 'fp2':batch[1].astype(np.float32), 'fp3':batch[1].astype(np.float32), 'ave':batch[1].astype(np.float32)})
        test_loss[ibatch] = result[0]
        test_acc[ibatch] = result[-1]
    return np.mean(test_loss), np.mean(test_acc)

def train_eval(model, train_set):
    """Evaluate on training set.
    Parameters
    -----------------
    model : keras.engine.training.Model object
        The training MI-Net with deep supervision model.
    train_set : list
        A list of training set contains all training bags features and labels.
    Returns
    -----------------
    test_loss : float
        Mean loss of evaluating on traing set.
    test_acc : float
        Mean accuracy of evaluating on testing set.
    """
    num_train_batch = len(train_set)
    train_loss = np.zeros((num_train_batch, 1), dtype=np.float32)
    train_acc = np.zeros((num_train_batch, 1), dtype=np.float32)
    shuffle(train_set)
    for ibatch, batch in enumerate(train_set):
        result = model.train_on_batch({'input':batch[0].astype(np.float32)}, {'fp1':batch[1].astype(np.float32), 'fp2':batch[1].astype(np.float32), 'fp3':batch[1].astype(np.float32), 'ave':batch[1].astype(np.float32)})
        train_loss[ibatch] = result[0]
        train_acc[ibatch] = result[-1]
    return np.mean(train_loss), np.mean(train_acc)

def MI_Net_with_DS(dataset):
    """Train and evaluate on MI-Net with deep supervision.
    Parameters
    -----------------
    dataset : dict
        A dictionary contains all dataset information. We split train/test by keys.
    Returns
    -----------------
    test_acc : float
        Testing accuracy of MI-Net with deep supervision.
    """
    weight_decay=0.005
    init_lr=5e-4
    pooling_mode='max'
    momentum=0.9
    max_epoch=50
    # load data and convert type
    train_bags = dataset['train']
    test_bags = dataset['test']

    # convert bag to batch
    train_set = convertToBatch(train_bags)
    test_set = convertToBatch(test_bags)
    dimension = train_set[0][0].shape[1]
    weight = [1.0, 1.0, 1.0, 0.0]

    # data: instance feature, n*d, n = number of training instance
    data_input = Input(shape=(dimension,), dtype='float32', name='input')

    # fully-connected
    fc1 = Dense(256, activation='relu', kernel_regularizer=l2(weight_decay))(data_input)
    fc2 = Dense(128, activation='relu', kernel_regularizer=l2(weight_decay))(fc1)
    fc3 = Dense(64, activation='relu', kernel_regularizer=l2(weight_decay))(fc2)

    # dropout
    dropout1 = Dropout(rate=0.5)(fc1)
    dropout2 = Dropout(rate=0.5)(fc2)
    dropout3 = Dropout(rate=0.5)(fc3)

    # features pooling
    fp1 = Feature_pooling(output_dim=1, kernel_regularizer=l2(weight_decay), pooling_mode=pooling_mode, name='fp1')(dropout1)
    fp2 = Feature_pooling(output_dim=1, kernel_regularizer=l2(weight_decay), pooling_mode=pooling_mode, name='fp2')(dropout2)
    fp3 = Feature_pooling(output_dim=1, kernel_regularizer=l2(weight_decay), pooling_mode=pooling_mode, name='fp3')(dropout3)

    # score average
    mg_ave =average([fp1,fp2,fp3], name='ave')

    model = Model(inputs=[data_input], outputs=[fp1, fp2, fp3, mg_ave])
    sgd = SGD(lr=init_lr, decay=1e-4, momentum=momentum, nesterov=True)
    model.compile(loss={'fp1':bag_loss, 'fp2':bag_loss, 'fp3':bag_loss, 'ave':bag_loss}, loss_weights={'fp1':weight[0], 'fp2':weight[1], 'fp3':weight[2], 'ave':weight[3]}, optimizer=sgd, metrics=[bag_accuracy])

    # train model
    t1 = time.time()
    num_batch = len(train_set)
    for epoch in range(max_epoch):
        train_loss, train_acc = train_eval(model, train_set)
        test_loss, test_acc = test_eval(model, test_set)
        print('epoch=', epoch, '  train_loss= {:.3f}'.format(train_loss), '  train_acc= {:.3f}'.format(train_acc), '  test_loss={:.3f}'.format(test_loss), '  test_acc= {:.3f}'.format(test_acc))
    t2 = time.time()
    print('run time:', (t2-t1) / 60, 'min')
    print('test_acc={:.3f}'.format(test_acc))

    return test_acc

In [10]:
# perform five times 10-fold cross=validation experiments
run = 5
n_folds = 10
acc = np.zeros((run, n_folds), dtype=float)
for irun in range(run):
    dataset = load_dataset('musk1', n_folds)
    for ifold in range(n_folds):
        print('run=', irun, '  fold=', ifold)
        acc[irun][ifold] = MI_Net_with_DS(dataset[ifold])
print('MI-Net with DS mean accuracy = ', np.mean(acc))
print('std = ', np.std(acc))

run= 0   fold= 0
epoch= 0   train_loss= 4.697   train_acc= 0.646   test_loss=3.608   test_acc= 0.700
epoch= 1   train_loss= 3.576   train_acc= 0.817   test_loss=3.062   test_acc= 0.900
epoch= 2   train_loss= 2.949   train_acc= 0.927   test_loss=3.044   test_acc= 0.900
epoch= 3   train_loss= 2.781   train_acc= 0.939   test_loss=2.858   test_acc= 0.900
epoch= 4   train_loss= 2.590   train_acc= 0.976   test_loss=2.990   test_acc= 0.900
epoch= 5   train_loss= 2.591   train_acc= 0.951   test_loss=2.754   test_acc= 1.000
epoch= 6   train_loss= 2.410   train_acc= 0.988   test_loss=2.781   test_acc= 0.900
epoch= 7   train_loss= 2.364   train_acc= 1.000   test_loss=2.655   test_acc= 1.000
epoch= 8   train_loss= 2.309   train_acc= 1.000   test_loss=2.594   test_acc= 1.000
epoch= 9   train_loss= 2.276   train_acc= 1.000   test_loss=2.758   test_acc= 0.900
epoch= 10   train_loss= 2.267   train_acc= 1.000   test_loss=2.562   test_acc= 1.000
epoch= 11   train_loss= 2.248   train_acc= 1.000   test_lo

epoch= 46   train_loss= 1.776   train_acc= 1.000   test_loss=2.678   test_acc= 0.800
epoch= 47   train_loss= 1.760   train_acc= 1.000   test_loss=2.742   test_acc= 0.800
epoch= 48   train_loss= 1.755   train_acc= 1.000   test_loss=2.752   test_acc= 0.800
epoch= 49   train_loss= 1.741   train_acc= 1.000   test_loss=2.751   test_acc= 0.800
run time: 1.3564526200294496 min
test_acc=0.800
run= 0   fold= 2
epoch= 0   train_loss= 4.495   train_acc= 0.699   test_loss=3.243   test_acc= 0.889
epoch= 1   train_loss= 3.333   train_acc= 0.867   test_loss=3.100   test_acc= 1.000
epoch= 2   train_loss= 2.935   train_acc= 0.952   test_loss=2.785   test_acc= 1.000
epoch= 3   train_loss= 2.819   train_acc= 0.964   test_loss=2.667   test_acc= 1.000
epoch= 4   train_loss= 2.698   train_acc= 0.964   test_loss=2.637   test_acc= 1.000
epoch= 5   train_loss= 2.569   train_acc= 0.976   test_loss=2.508   test_acc= 1.000
epoch= 6   train_loss= 2.531   train_acc= 0.988   test_loss=2.448   test_acc= 1.000
epoch= 

epoch= 42   train_loss= 1.805   train_acc= 1.000   test_loss=2.073   test_acc= 1.000
epoch= 43   train_loss= 1.794   train_acc= 1.000   test_loss=2.052   test_acc= 1.000
epoch= 44   train_loss= 1.781   train_acc= 1.000   test_loss=2.047   test_acc= 1.000
epoch= 45   train_loss= 1.776   train_acc= 1.000   test_loss=2.029   test_acc= 1.000
epoch= 46   train_loss= 1.760   train_acc= 1.000   test_loss=2.007   test_acc= 1.000
epoch= 47   train_loss= 1.754   train_acc= 1.000   test_loss=2.004   test_acc= 1.000
epoch= 48   train_loss= 1.743   train_acc= 1.000   test_loss=1.992   test_acc= 1.000
epoch= 49   train_loss= 1.732   train_acc= 1.000   test_loss=1.968   test_acc= 1.000
run time: 1.2091976523399353 min
test_acc=1.000
run= 0   fold= 4
epoch= 0   train_loss= 4.329   train_acc= 0.747   test_loss=4.795   test_acc= 0.333
epoch= 1   train_loss= 3.264   train_acc= 0.904   test_loss=4.177   test_acc= 0.667
epoch= 2   train_loss= 2.806   train_acc= 0.976   test_loss=4.303   test_acc= 0.667
epo

epoch= 38   train_loss= 1.842   train_acc= 1.000   test_loss=2.998   test_acc= 0.889
epoch= 39   train_loss= 1.833   train_acc= 1.000   test_loss=3.003   test_acc= 0.889
epoch= 40   train_loss= 1.828   train_acc= 1.000   test_loss=2.969   test_acc= 0.889
epoch= 41   train_loss= 1.818   train_acc= 1.000   test_loss=2.956   test_acc= 0.889
epoch= 42   train_loss= 1.806   train_acc= 1.000   test_loss=2.971   test_acc= 0.889
epoch= 43   train_loss= 1.791   train_acc= 1.000   test_loss=2.968   test_acc= 0.889
epoch= 44   train_loss= 1.780   train_acc= 1.000   test_loss=2.941   test_acc= 0.889
epoch= 45   train_loss= 1.771   train_acc= 1.000   test_loss=2.940   test_acc= 0.889
epoch= 46   train_loss= 1.762   train_acc= 1.000   test_loss=2.942   test_acc= 0.889
epoch= 47   train_loss= 1.750   train_acc= 1.000   test_loss=2.935   test_acc= 0.889
epoch= 48   train_loss= 1.739   train_acc= 1.000   test_loss=2.934   test_acc= 0.889
epoch= 49   train_loss= 1.739   train_acc= 1.000   test_loss=2.92

epoch= 34   train_loss= 1.880   train_acc= 1.000   test_loss=4.390   test_acc= 0.667
epoch= 35   train_loss= 1.866   train_acc= 1.000   test_loss=4.461   test_acc= 0.667
epoch= 36   train_loss= 1.855   train_acc= 1.000   test_loss=4.388   test_acc= 0.667
epoch= 37   train_loss= 1.841   train_acc= 1.000   test_loss=4.349   test_acc= 0.667
epoch= 38   train_loss= 1.827   train_acc= 1.000   test_loss=4.373   test_acc= 0.667
epoch= 39   train_loss= 1.817   train_acc= 1.000   test_loss=4.385   test_acc= 0.667
epoch= 40   train_loss= 1.804   train_acc= 1.000   test_loss=4.400   test_acc= 0.667
epoch= 41   train_loss= 1.799   train_acc= 1.000   test_loss=4.332   test_acc= 0.667
epoch= 42   train_loss= 1.788   train_acc= 1.000   test_loss=4.353   test_acc= 0.667
epoch= 43   train_loss= 1.776   train_acc= 1.000   test_loss=4.289   test_acc= 0.667
epoch= 44   train_loss= 1.762   train_acc= 1.000   test_loss=4.243   test_acc= 0.667
epoch= 45   train_loss= 1.753   train_acc= 1.000   test_loss=4.32

...

epoch= 12   train_loss= 2.222   train_acc= 1.000   test_loss=2.343   test_acc= 1.000
epoch= 13   train_loss= 2.191   train_acc= 1.000   test_loss=2.321   test_acc= 1.000
epoch= 14   train_loss= 2.154   train_acc= 1.000   test_loss=2.313   test_acc= 1.000
epoch= 15   train_loss= 2.149   train_acc= 1.000   test_loss=2.301   test_acc= 1.000
epoch= 16   train_loss= 2.128   train_acc= 1.000   test_loss=2.273   test_acc= 1.000
epoch= 17   train_loss= 2.125   train_acc= 1.000   test_loss=2.231   test_acc= 1.000
epoch= 18   train_loss= 2.081   train_acc= 1.000   test_loss=2.216   test_acc= 1.000
epoch= 19   train_loss= 2.083   train_acc= 1.000   test_loss=2.202   test_acc= 1.000
epoch= 20   train_loss= 2.067   train_acc= 1.000   test_loss=2.184   test_acc= 1.000
epoch= 21   train_loss= 2.045   train_acc= 1.000   test_loss=2.167   test_acc= 1.000
epoch= 22   train_loss= 2.036   train_acc= 1.000   test_loss=2.149   test_acc= 1.000
epoch= 23   train_loss= 2.036   train_acc= 1.000   test_loss=2.14

epoch= 8   train_loss= 2.323   train_acc= 1.000   test_loss=3.088   test_acc= 0.889
epoch= 9   train_loss= 2.328   train_acc= 1.000   test_loss=3.091   test_acc= 0.778
epoch= 10   train_loss= 2.353   train_acc= 0.988   test_loss=3.253   test_acc= 0.778
epoch= 11   train_loss= 2.263   train_acc= 0.988   test_loss=3.033   test_acc= 0.778
epoch= 12   train_loss= 2.225   train_acc= 1.000   test_loss=3.114   test_acc= 0.778
epoch= 13   train_loss= 2.229   train_acc= 1.000   test_loss=2.975   test_acc= 0.889
epoch= 14   train_loss= 2.191   train_acc= 1.000   test_loss=3.202   test_acc= 0.778
epoch= 15   train_loss= 2.176   train_acc= 1.000   test_loss=2.926   test_acc= 0.778
epoch= 16   train_loss= 2.149   train_acc= 1.000   test_loss=2.945   test_acc= 0.778
epoch= 17   train_loss= 2.124   train_acc= 1.000   test_loss=2.925   test_acc= 0.778
epoch= 18   train_loss= 2.116   train_acc= 1.000   test_loss=2.870   test_acc= 0.889
epoch= 19   train_loss= 2.106   train_acc= 1.000   test_loss=3.021 

epoch= 4   train_loss= 2.838   train_acc= 0.952   test_loss=2.817   test_acc= 1.000
epoch= 5   train_loss= 2.610   train_acc= 0.976   test_loss=2.784   test_acc= 0.889
epoch= 6   train_loss= 2.515   train_acc= 0.988   test_loss=2.673   test_acc= 1.000
epoch= 7   train_loss= 2.433   train_acc= 0.988   test_loss=2.621   test_acc= 1.000
epoch= 8   train_loss= 2.388   train_acc= 0.988   test_loss=2.589   test_acc= 1.000
epoch= 9   train_loss= 2.388   train_acc= 0.988   test_loss=2.543   test_acc= 1.000
epoch= 10   train_loss= 2.311   train_acc= 1.000   test_loss=2.457   test_acc= 1.000
epoch= 11   train_loss= 2.315   train_acc= 0.988   test_loss=2.456   test_acc= 1.000
epoch= 12   train_loss= 2.260   train_acc= 1.000   test_loss=2.410   test_acc= 1.000
epoch= 13   train_loss= 2.253   train_acc= 1.000   test_loss=2.398   test_acc= 1.000
epoch= 14   train_loss= 2.212   train_acc= 1.000   test_loss=2.350   test_acc= 1.000
epoch= 15   train_loss= 2.198   train_acc= 1.000   test_loss=2.339   te

epoch= 0   train_loss= 4.349   train_acc= 0.675   test_loss=4.071   test_acc= 0.778
epoch= 1   train_loss= 3.517   train_acc= 0.855   test_loss=3.668   test_acc= 0.778
epoch= 2   train_loss= 2.958   train_acc= 0.952   test_loss=3.866   test_acc= 0.778
epoch= 3   train_loss= 2.739   train_acc= 0.952   test_loss=3.498   test_acc= 0.778
epoch= 4   train_loss= 2.633   train_acc= 0.976   test_loss=3.398   test_acc= 0.778
epoch= 5   train_loss= 2.480   train_acc= 0.988   test_loss=3.388   test_acc= 0.778
epoch= 6   train_loss= 2.428   train_acc= 0.988   test_loss=3.458   test_acc= 0.889
epoch= 7   train_loss= 2.378   train_acc= 1.000   test_loss=3.345   test_acc= 0.778
epoch= 8   train_loss= 2.311   train_acc= 1.000   test_loss=3.404   test_acc= 0.778
epoch= 9   train_loss= 2.335   train_acc= 1.000   test_loss=3.350   test_acc= 0.778
epoch= 10   train_loss= 2.269   train_acc= 1.000   test_loss=3.317   test_acc= 0.778
epoch= 11   train_loss= 2.243   train_acc= 1.000   test_loss=3.404   test_a

epoch= 46   train_loss= 1.758   train_acc= 1.000   test_loss=2.699   test_acc= 0.778
epoch= 47   train_loss= 1.747   train_acc= 1.000   test_loss=2.683   test_acc= 0.778
epoch= 48   train_loss= 1.737   train_acc= 1.000   test_loss=2.665   test_acc= 0.778
epoch= 49   train_loss= 1.737   train_acc= 1.000   test_loss=2.669   test_acc= 0.778
run time: 1.343178351720174 min
test_acc=0.778
MI-Net with DS mean accuracy =  0.8822222262620926
std =  0.12047231613055681


## Bag-Space

*CkNN*

https://github.com/chlorochrule/cknn/blob/master/cknn/cknn.py

In [ ]:
import pandas as pd
X = pd.read_table("./clean2.data") #pd.read_csv("sample_data/mnist_test.csv") 

In [ ]:
from cknn import cknneighbors_graph

#ckng = cknneighbors_graph(X, n_neighbors=5, delta=1.0)

In [ ]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.manifold import SpectralEmbedding
import matplotlib.pyplot as plt
from matplotlib import offsetbox
import seaborn as sns

from cknn import cknneighbors_graph

sns.set()


def plot2d_label(X, title=None):
    digits = load_digits()
    y = digits.target
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)

    plt.figure()
    ax = plt.subplot(111)
    for i in range(X.shape[0]):
        plt.text(X[i, 0], X[i, 1], str(digits.target[i]),
                 color=plt.cm.Set1(y[i] / 10.),
                 fontdict={'weight': 'bold', 'size': 9})

    
    plt.xticks([])
    plt.yticks([])
    if title is not None:
        plt.title(title)
    plt.show()


def main():
    data = X
    print(data)
    n_neighbors = 2

    model_normal = SpectralEmbedding(n_components=2, n_neighbors=n_neighbors)
    y_normal = model_normal.fit_transform(data)
    plot2d_label(y_normal)

    #ckng = cknneighbors_graph(data, n_neighbors=n_neighbors, delta=1.5)
    #model_cknn = SpectralEmbedding(n_components=2, affinity='precomputed')
    #y_cknn = model_cknn.fit_transform(ckng.toarray())
    #plot2d_label(y_cknn)

main()

     MUSK-211,211_1+1,46,-108,-60,-69,-117,49,38,-161,-8,5,-323,-220,-113,-299,-283,-307,-31,-106,-227,-42,-59,-22,-67,189,81,17,-27,-89,-67,105,-116,124,-106,5,-120,63,-165,40,-27,68,-44,98,-33,-314,-282,-335,-144,-13,-197,-2,-144,-13,-11,-131,108,-43,42,-151,-4,8,-102,51,-15,108,-135,59,-166,20,-20,23,-48,-68,-299,-256,-97,-183,-24,-271,-229,-177,-6,0,-129,112,15,36,-66,-54,-75,132,-188,119,-120,-312,23,-55,-53,-26,-71,41,-55,148,-247,-306,-308,-230,-166,-35,-205,-280,-239,-53,-10,-23,25,-5,163,61,59,-39,92,72,113,-107,80,25,-27,81,-114,-187,45,-118,-75,-182,-234,-19,12,-13,-41,-119,-149,70,17,-20,-177,-101,-116,-14,-50,24,-81,-125,-114,-44,128,3,-244,-308,52,-7,39,126,156,-50,-112,96,1.
0     MUSK-211,211_1+10,41,-188,-145,22,-117,-6,57,-...                                                                                                                                                                                                                                                      

ValueError: ignored

In [1]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.manifold import SpectralEmbedding
import matplotlib.pyplot as plt
from matplotlib import offsetbox
import seaborn as sns

sns.set()


def plot2d_label(X, title=None):
    y = X[1]
    x_min, x_max = np.min(X[0], 0), np.max(X[0], 0)
    X = (X - x_min) / (x_max - x_min)

    plt.figure()
    plt.xticks([])
    plt.yticks([])
    if title is not None:
        plt.title(title)
    plt.show()


def main():
    data = X
    n_neighbors = 10

    model_normal = SpectralEmbedding(n_components=2, n_neighbors=n_neighbors)
    y_normal = model_normal.fit_transform(data)
    #plot2d_label(y_normal)

    ckng = cknneighbors_graph(data, n_neighbors=n_neighbors, delta=1.5)
    model_cknn = SpectralEmbedding(n_components=2, affinity='precomputed')
    y_cknn = model_cknn.fit_transform(ckng.toarray())
    #plot2d_label(y_cknn)
    print(y_cknn)

main()

NameError: name 'X' is not defined

## instance-Space


MI-SVM and mi-SVM

In [15]:
import misvm
from loader import parse_c45, bag_set
from __future__ import print_function, division
import numpy as np
from sklearn.model_selection import StratifiedKFold

In [ ]:
# Load list of C4.5 Examples
example_set = parse_c45('elephant')

# Get stats to normalize data
raw_data = np.array(example_set.to_float())
data_mean = np.average(raw_data, axis=0)
data_std  = np.std(raw_data, axis=0)
data_std[np.nonzero(data_std == 0.0)] = 1.0
def normalizer(ex):
    ex = np.array(ex)
    normed = ((ex - data_mean) / data_std)
    # The ...[:, 2:-1] removes first two columns and last column,
    # which are the bag/instance ids and class label, as part of the
    # normalization process
    return normed[2:-1]


# Group examples into bags
bagset = bag_set(example_set)

# Convert bags to NumPy arrays
bags = [np.array(b.to_float(normalizer)) for b in bagset]
labels = np.array([b.label for b in bagset], dtype=float)
# Convert 0/1 labels to -1/1 labels
labels = 2 * labels - 1

# Spilt dataset arbitrarily to train/test sets
train_bags = bags[10:]
train_labels = labels[10:]
test_bags = bags[:10]
test_labels = labels[:10]

# Construct classifiers
classifiers = {}

# MISVM   : the MI-SVM algorithm of Andrews, Tsochantaridis, & Hofmann (2002)
# miSVM   : the mi-SVM algorithm of Andrews, Tsochantaridis, & Hofmann (2002)

#  : the semi-supervised learning approach of Zhou & Xu (2007)
#     : the MI classification algorithm of Mangasarian & Wild (2008)
# sMIL    : sparse MIL (Bunescu & Mooney, 2007)
# stMIL   : sparse, transductive  MIL (Bunescu & Mooney, 2007)

classifiers['MissSVM'] = misvm.MissSVM(kernel='linear', C=1.0, max_iters=20)
classifiers['sbMIL'] = misvm.sbMIL(kernel='linear', eta=0.1, C=1e2)
classifiers['SIL'] = misvm.SIL(kernel='linear', C=1.0)
classifiers['STK'] = misvm.STK(kernel='linear', C=1.0)
classifiers['NSK'] = misvm.NSK(kernel='linear', C=1.0)
classifiers['MICA'] = misvm.MICA(kernel='linear', C=1.0)

# Train/Evaluate classifiers
accuracies = {}

bags = np.array(bags,dtype=object)
labels = np.array(labels,dtype=int)
fold = StratifiedKFold(n_splits=5, shuffle=False, random_state=None)
for algorithm, classifier in classifiers.items():
    nums = 1
    for train_index, test_index in fold.split(bags,labels):
        X_train, X_test = bags[train_index], bags[test_index]
        y_train, y_test = labels[train_index], labels[test_index]

        classifier.fit(X_train, y_train)
        predictions = classifier.predict(X_test)
        accuracies[algorithm + " " + str(nums)] = {"acc":np.average(y_test == np.sign(predictions)),"kfold":nums}
        nums+=1

for algorithm, item in accuracies.items():
    print('\n%s, fold:%s Accuracy: %.f%%' % (algorithm,str(item["kfold"]), 100 * item["acc"]))

Non-random start...
     pcost       dcost       gap    pres   dres
 0: -7.6545e+02 -2.1590e+01  1e+04  1e+02  6e-12
 1: -2.0029e+01 -2.1396e+01  2e+02  2e+00  6e-12
 2: -7.3228e+00 -1.7055e+01  3e+01  2e-01  6e-13
 3: -6.0847e+00 -1.2518e+01  1e+01  8e-02  2e-13
 4: -5.6381e+00 -8.7330e+00  4e+00  2e-02  9e-14
 5: -5.9445e+00 -6.7087e+00  9e-01  4e-03  8e-14
 6: -6.1173e+00 -6.3318e+00  2e-01  7e-04  8e-14
 7: -6.1738e+00 -6.2387e+00  7e-02  1e-04  8e-14
 8: -6.1936e+00 -6.2095e+00  2e-02  2e-05  8e-14
 9: -6.1989e+00 -6.2025e+00  4e-03  5e-06  8e-14
10: -6.2002e+00 -6.2007e+00  5e-04  3e-07  8e-14
11: -6.2004e+00 -6.2005e+00  4e-05  2e-08  8e-14
12: -6.2005e+00 -6.2005e+00  9e-07  5e-10  8e-14
Optimal solution found.

Iteration 1...
Linearizing constraints...
Computing slacks...
Linearizing...
Solving QP...
     pcost       dcost       gap    pres   dres
 0: -7.6319e+02 -2.1716e+01  1e+04  1e+02  6e-12
 1: -1.9330e+01 -2.1504e+01  2e+02  2e+00  6e-12
 2: -7.7071e+00 -1.7057e+01  4e+0

10: -6.4494e+00 -6.4579e+00  1e-02  5e-06  8e-14
11: -6.4523e+00 -6.4542e+00  2e-03  6e-07  8e-14
12: -6.4531e+00 -6.4533e+00  2e-04  5e-08  8e-14
13: -6.4532e+00 -6.4532e+00  6e-06  1e-09  8e-14
Optimal solution found.
delta obj ratio: 5.56e+04

Iteration 10...
Linearizing constraints...
Computing slacks...
Linearizing...
Solving QP...
     pcost       dcost       gap    pres   dres
 0: -7.6379e+02 -2.1651e+01  1e+04  1e+02  6e-12
 1: -2.0141e+01 -2.1442e+01  2e+02  2e+00  6e-12
 2: -7.3063e+00 -1.7063e+01  4e+01  3e-01  7e-13
 3: -5.8499e+00 -1.0788e+01  9e+00  5e-02  2e-13
 4: -5.9296e+00 -7.0741e+00  1e+00  5e-03  8e-14
 5: -6.0816e+00 -6.7132e+00  8e-01  3e-03  7e-14
 6: -6.1944e+00 -6.5015e+00  4e-01  7e-04  7e-14
 7: -6.2252e+00 -6.4395e+00  3e-01  3e-04  7e-14
 8: -6.2832e+00 -6.3432e+00  7e-02  6e-05  8e-14
 9: -6.3030e+00 -6.3141e+00  1e-02  6e-06  8e-14
10: -6.3071e+00 -6.3086e+00  2e-03  7e-07  8e-14
11: -6.3076e+00 -6.3079e+00  3e-04  1e-07  8e-14
12: -6.3078e+00 -6.3078e+

 4: -6.0787e+00 -7.9035e+00  3e+00  1e-02  8e-14
 5: -6.3376e+00 -6.9004e+00  8e-01  3e-03  8e-14
 6: -6.4388e+00 -6.6485e+00  3e-01  9e-04  7e-14
 7: -6.4622e+00 -6.6029e+00  2e-01  5e-04  7e-14
 8: -6.4963e+00 -6.5433e+00  6e-02  1e-04  7e-14
 9: -6.5091e+00 -6.5224e+00  2e-02  3e-05  8e-14
10: -6.5135e+00 -6.5158e+00  3e-03  4e-06  8e-14
11: -6.5144e+00 -6.5146e+00  3e-04  3e-07  8e-14
12: -6.5145e+00 -6.5145e+00  9e-06  8e-09  9e-14
13: -6.5145e+00 -6.5145e+00  1e-07  1e-10  9e-14
Optimal solution found.
delta obj ratio: 1.62e+05

Iteration 20...
Linearizing constraints...
Computing slacks...
Linearizing...
Solving QP...
     pcost       dcost       gap    pres   dres
 0: -7.6457e+02 -2.1668e+01  1e+04  1e+02  6e-12
 1: -2.0437e+01 -2.1459e+01  2e+02  2e+00  6e-12
 2: -7.6851e+00 -1.7246e+01  4e+01  3e-01  8e-13
 3: -5.7914e+00 -1.1978e+01  1e+01  6e-02  2e-13
 4: -6.0541e+00 -7.2980e+00  2e+00  8e-03  8e-14
 5: -6.2733e+00 -6.8463e+00  8e-01  3e-03  7e-14
 6: -6.3850e+00 -6.6235e+

Non-random start...
     pcost       dcost       gap    pres   dres
 0: -7.3155e+02 -2.2279e+01  1e+04  1e+02  6e-12
 1: -1.5371e+01 -2.2123e+01  2e+02  2e+00  6e-12
 2: -6.6710e+00 -1.6864e+01  3e+01  2e-01  6e-13
 3: -5.6956e+00 -1.2579e+01  1e+01  8e-02  3e-13
 4: -5.3435e+00 -8.0179e+00  3e+00  1e-02  9e-14
 5: -5.7209e+00 -6.3130e+00  7e-01  2e-03  7e-14
 6: -5.8135e+00 -6.1591e+00  4e-01  1e-03  7e-14
 7: -5.9025e+00 -6.0131e+00  1e-01  3e-04  7e-14
 8: -5.9337e+00 -5.9663e+00  3e-02  6e-05  7e-14
 9: -5.9449e+00 -5.9509e+00  6e-03  5e-06  8e-14
10: -5.9472e+00 -5.9480e+00  8e-04  6e-07  8e-14
11: -5.9476e+00 -5.9476e+00  5e-05  4e-08  8e-14
12: -5.9476e+00 -5.9476e+00  5e-07  4e-10  8e-14
Optimal solution found.

Iteration 1...
Linearizing constraints...
Computing slacks...
Linearizing...
Solving QP...
     pcost       dcost       gap    pres   dres
 0: -7.3442e+02 -2.2296e+01  1e+04  1e+02  6e-12
 1: -1.6433e+01 -2.2136e+01  2e+02  2e+00  6e-12
 2: -7.0312e+00 -1.7135e+01  3e+0

 3: -5.6152e+00 -1.1650e+01  1e+01  8e-02  3e-13
 4: -5.5633e+00 -6.9913e+00  2e+00  8e-03  8e-14
 5: -5.7965e+00 -6.4573e+00  9e-01  3e-03  7e-14
 6: -5.8884e+00 -6.2759e+00  5e-01  1e-03  7e-14
 7: -5.9714e+00 -6.1194e+00  2e-01  3e-04  7e-14
 8: -6.0126e+00 -6.0526e+00  4e-02  1e-05  8e-14
 9: -6.0250e+00 -6.0355e+00  1e-02  3e-06  7e-14
10: -6.0288e+00 -6.0306e+00  2e-03  4e-07  7e-14
11: -6.0295e+00 -6.0296e+00  8e-05  9e-09  8e-14
12: -6.0296e+00 -6.0296e+00  2e-06  2e-10  8e-14
Optimal solution found.
delta obj ratio: 2.38e+05

Iteration 11...
Linearizing constraints...
Computing slacks...
Linearizing...
Solving QP...
     pcost       dcost       gap    pres   dres
 0: -7.3169e+02 -2.2040e+01  1e+04  1e+02  7e-12
 1: -1.7411e+01 -2.1877e+01  2e+02  2e+00  6e-12
 2: -6.5968e+00 -1.7174e+01  3e+01  2e-01  7e-13
 3: -5.6721e+00 -1.0654e+01  9e+00  5e-02  2e-13
 4: -5.8251e+00 -7.0889e+00  2e+00  8e-03  8e-14
 5: -6.0482e+00 -6.5174e+00  7e-01  3e-03  7e-14
 6: -6.1516e+00 -6.3156e+

...

Non-random start...
     pcost       dcost       gap    pres   dres
 0: -7.4224e+02 -2.2233e+01  1e+04  1e+02  6e-12
 1: -2.2481e+01 -2.2074e+01  3e+02  3e+00  6e-12
 2: -7.1162e+00 -1.8096e+01  4e+01  2e-01  5e-13
 3: -5.8044e+00 -1.2538e+01  1e+01  8e-02  2e-13
 4: -5.3548e+00 -8.4543e+00  4e+00  2e-02  8e-14
 5: -5.6817e+00 -6.4953e+00  9e-01  4e-03  8e-14
 6: -5.8216e+00 -6.1754e+00  4e-01  1e-03  7e-14
 7: -5.9189e+00 -5.9966e+00  8e-02  2e-04  7e-14
 8: -5.9396e+00 -5.9643e+00  3e-02  4e-05  7e-14
 9: -5.9472e+00 -5.9533e+00  6e-03  9e-06  8e-14
10: -5.9493e+00 -5.9505e+00  1e-03  7e-07  8e-14
11: -5.9498e+00 -5.9499e+00  3e-05  1e-08  8e-14
12: -5.9498e+00 -5.9498e+00  9e-07  3e-10  8e-14
Optimal solution found.

Iteration 1...
Linearizing constraints...
Computing slacks...
Linearizing...
Solving QP...
     pcost       dcost       gap    pres   dres
 0: -7.4103e+02 -2.2376e+01  1e+04  1e+02  6e-12
 1: -2.1710e+01 -2.2195e+01  3e+02  3e+00  6e-12
 2: -7.1705e+00 -1.7981e+01  4e+0

Solving QP...
     pcost       dcost       gap    pres   dres
 0: -7.4405e+02 -2.2016e+01  1e+04  1e+02  6e-12
 1: -2.0760e+01 -2.1852e+01  3e+02  2e+00  6e-12
 2: -7.0947e+00 -1.7700e+01  4e+01  3e-01  7e-13
 3: -5.8982e+00 -1.2061e+01  1e+01  8e-02  2e-13
 4: -5.8582e+00 -7.8788e+00  3e+00  2e-02  9e-14
 5: -6.0661e+00 -6.9670e+00  1e+00  7e-03  7e-14
 6: -6.2059e+00 -6.5055e+00  4e-01  2e-03  7e-14
 7: -6.2475e+00 -6.4145e+00  2e-01  8e-04  7e-14
 8: -6.2762e+00 -6.3576e+00  1e-01  3e-04  7e-14
 9: -6.2954e+00 -6.3229e+00  3e-02  6e-05  8e-14
10: -6.3043e+00 -6.3098e+00  7e-03  1e-05  8e-14
11: -6.3062e+00 -6.3071e+00  1e-03  2e-06  7e-14
12: -6.3065e+00 -6.3066e+00  7e-05  7e-08  8e-14
13: -6.3066e+00 -6.3066e+00  1e-06  7e-10  8e-14
Optimal solution found.
delta obj ratio: 3.85e+05

Iteration 11...
Linearizing constraints...
Computing slacks...
Linearizing...
Solving QP...
     pcost       dcost       gap    pres   dres
 0: -7.4004e+02 -2.2049e+01  1e+04  1e+02  6e-12
 1: -2.1005e

11: -6.2038e+00 -6.2044e+00  7e-04  3e-07  8e-14
12: -6.2040e+00 -6.2041e+00  7e-05  3e-08  8e-14
13: -6.2041e+00 -6.2041e+00  4e-06  9e-10  8e-14
Optimal solution found.
delta obj ratio: 7.37e+04

Iteration 20...
Linearizing constraints...
Computing slacks...
Linearizing...
Solving QP...
     pcost       dcost       gap    pres   dres
 0: -7.4264e+02 -2.1927e+01  1e+04  1e+02  6e-12
 1: -1.9084e+01 -2.1754e+01  2e+02  2e+00  6e-12
 2: -7.0670e+00 -1.7226e+01  4e+01  2e-01  7e-13
 3: -5.8361e+00 -1.1266e+01  1e+01  6e-02  2e-13
 4: -5.8504e+00 -7.0797e+00  2e+00  4e-03  8e-14
 5: -6.0671e+00 -6.6661e+00  7e-01  2e-03  7e-14
 6: -6.1882e+00 -6.4419e+00  3e-01  6e-04  7e-14
 7: -6.2382e+00 -6.3626e+00  1e-01  2e-04  7e-14
 8: -6.2734e+00 -6.3094e+00  4e-02  5e-05  7e-14
 9: -6.2840e+00 -6.2940e+00  1e-02  2e-06  8e-14
10: -6.2878e+00 -6.2895e+00  2e-03  3e-07  8e-14
11: -6.2884e+00 -6.2887e+00  2e-04  4e-08  8e-14
12: -6.2885e+00 -6.2885e+00  9e-06  1e-09  8e-14
13: -6.2885e+00 -6.2885e+

Non-random start...
     pcost       dcost       gap    pres   dres
 0: -7.4639e+02 -2.2025e+01  1e+04  1e+02  6e-12
 1: -1.5421e+01 -2.1872e+01  2e+02  2e+00  6e-12
 2: -6.7026e+00 -1.6825e+01  3e+01  2e-01  6e-13
 3: -5.7478e+00 -1.2292e+01  1e+01  7e-02  3e-13
 4: -5.5091e+00 -7.9441e+00  3e+00  1e-02  9e-14
 5: -5.8840e+00 -6.5021e+00  7e-01  2e-03  7e-14
 6: -6.0486e+00 -6.2145e+00  2e-01  5e-04  7e-14
 7: -6.0930e+00 -6.1458e+00  6e-02  1e-04  7e-14
 8: -6.1097e+00 -6.1216e+00  1e-02  2e-05  8e-14
 9: -6.1143e+00 -6.1155e+00  1e-03  1e-06  8e-14
10: -6.1148e+00 -6.1149e+00  9e-05  1e-07  8e-14
11: -6.1149e+00 -6.1149e+00  5e-06  6e-09  8e-14
Optimal solution found.

Iteration 1...
Linearizing constraints...
Computing slacks...
Linearizing...
Solving QP...
     pcost       dcost       gap    pres   dres
 0: -7.4452e+02 -2.2211e+01  1e+04  1e+02  7e-12
 1: -1.6509e+01 -2.2049e+01  2e+02  2e+00  6e-12
 2: -6.6547e+00 -1.7061e+01  4e+01  2e-01  8e-13
 3: -5.3105e+00 -1.1548e+01  1e+0

Solving QP...
     pcost       dcost       gap    pres   dres
 0: -7.4566e+02 -2.2103e+01  1e+04  1e+02  6e-12
 1: -1.5738e+01 -2.1940e+01  2e+02  2e+00  6e-12
 2: -6.5516e+00 -1.6786e+01  3e+01  2e-01  7e-13
 3: -5.4575e+00 -1.0517e+01  9e+00  4e-02  2e-13
 4: -5.7264e+00 -6.6968e+00  1e+00  6e-03  8e-14
 5: -5.9554e+00 -6.2955e+00  5e-01  2e-03  7e-14
 6: -6.0235e+00 -6.1878e+00  2e-01  8e-04  7e-14
 7: -6.0576e+00 -6.1365e+00  1e-01  2e-04  7e-14
 8: -6.0792e+00 -6.1067e+00  3e-02  7e-05  7e-14
 9: -6.0884e+00 -6.0946e+00  7e-03  1e-05  8e-14
10: -6.0908e+00 -6.0916e+00  1e-03  1e-06  8e-14
11: -6.0911e+00 -6.0911e+00  3e-05  3e-08  8e-14
12: -6.0911e+00 -6.0911e+00  1e-06  1e-09  8e-14
Optimal solution found.
delta obj ratio: 1.28e+05

Iteration 11...
Linearizing constraints...
Computing slacks...
Linearizing...
Solving QP...
     pcost       dcost       gap    pres   dres
 0: -7.4490e+02 -2.2030e+01  1e+04  1e+02  8e-12
 1: -1.5934e+01 -2.1874e+01  2e+02  2e+00  6e-12
 2: -6.4221e

### mi-Net

In [1]:
import sys
import time
from random import shuffle
import numpy as np
import argparse

from keras.models import Model
from keras.optimizers import SGD
from keras.regularizers import l2
from keras.layers import Input, Dense, Layer, Dropout

from mil_nets.dataset import load_dataset
from mil_nets.layer import Score_pooling
from mil_nets.metrics import bag_accuracy
from mil_nets.objectives import bag_loss
from mil_nets.utils import convertToBatch

In [22]:
def test_eval(model, test_set):
    """Evaluate on testing set.
    Parameters
    -----------------
    model : keras.engine.training.Model object
        The training mi-Net model.
    test_set : list
        A list of testing set contains all training bags features and labels.
    Returns
    -----------------
    test_loss : float
        Mean loss of evaluating on testing set.
    test_acc : float
        Mean accuracy of evaluating on testing set.
    """
    num_test_batch = len(test_set)
    test_loss = np.zeros((num_test_batch, 1), dtype=np.float32)
    test_acc = np.zeros((num_test_batch, 1), dtype=np.float32)
    for ibatch, batch in enumerate(test_set):
        result = model.test_on_batch({'input':batch[0].astype(np.float32)}, {'sp':batch[1].astype(np.float32)})
        test_loss[ibatch] = result[0]
        test_acc[ibatch] = result[1]
    return np.mean(test_loss), np.mean(test_acc)

def train_eval(model, train_set):
    """Evaluate on training set.
    Parameters
    -----------------
    model : keras.engine.training.Model object
        The training mi-Net model.
    train_set : list
        A list of training set contains all training bags features and labels.
    Returns
    -----------------
    test_loss : float
        Mean loss of evaluating on traing set.
    test_acc : float
        Mean accuracy of evaluating on testing set.
    """
    num_train_batch = len(train_set)
    train_loss = np.zeros((num_train_batch, 1), dtype=np.float32)
    train_acc = np.zeros((num_train_batch, 1), dtype=np.float32)
    shuffle(train_set)
    for ibatch, batch in enumerate(train_set):
        result = model.train_on_batch({'input':batch[0].astype(np.float32)}, {'sp':batch[1].astype(np.float32)})
        train_loss[ibatch] = result[0]
        train_acc[ibatch] = result[1]
    return np.mean(train_loss), np.mean(train_acc)

def mi_Net(dataset):
    weight_decay=0.005
    init_lr=5e-4
    pooling_mode='max'
    momentum=0.9
    max_epoch=50
    """Train and evaluate on mi-Net.
    Parameters
    -----------------
    dataset : dict
        A dictionary contains all dataset information. We split train/test by keys.
    Returns
    -----------------
    test_acc : float
        Testing accuracy of mi-Net.
    """
    # load data and convert type
    train_bags = dataset['train']
    test_bags = dataset['test']

    # convert bag to batch
    train_set = convertToBatch(train_bags)
    test_set = convertToBatch(test_bags)
    dimension = train_set[0][0].shape[1]

    # data: instance feature, n*d, n = number of training instance
    data_input = Input(shape=(dimension,), dtype='float32', name='input')

    # fully-connected
    fc1 = Dense(256, activation='relu', kernel_regularizer=l2(weight_decay))(data_input)
    fc2 = Dense(128, activation='relu', kernel_regularizer=l2(weight_decay))(fc1)
    fc3 = Dense(64, activation='relu', kernel_regularizer=l2(weight_decay))(fc2)

    # dropout
    dropout = Dropout(rate=0.5)(fc3)

    # score pooling
    sp = Score_pooling(output_dim=1, kernel_regularizer=l2(weight_decay), pooling_mode=pooling_mode, name='sp')(dropout)

    model = Model(inputs=[data_input], outputs=[sp])
    sgd = SGD(lr=init_lr, decay=1e-4, momentum=momentum, nesterov=True)
    model.compile(loss=bag_loss, optimizer=sgd, metrics=[bag_accuracy])

    # train model
    t1 = time.time()
    num_batch = len(train_set)
    for epoch in range(max_epoch):
        train_loss, train_acc = train_eval(model, train_set)
        test_loss, test_acc = test_eval(model, test_set)
        print('epoch=', epoch, '  train_loss= {:.3f}'.format(train_loss), '  train_acc= {:.3f}'.format(train_acc), '  test_loss={:.3f}'.format(test_loss), '  test_acc= {:.3f}'.format(test_acc))
    t2 = time.time()
    print('run time:', (t2-t1) / 60.0, 'min')
    print('test_acc={:.3f}'.format(test_acc))

    return test_acc


In [23]:
# perform five times 10-fold cross-validation experiments
run = 5
n_folds = 10
acc = np.zeros((run, n_folds), dtype=np.float32)
for irun in range(run):
    dataset = load_dataset('musk1', n_folds)
    for ifold in range(n_folds):
        print('run=', irun, '  fold=', ifold)
        acc[irun][ifold] = mi_Net(dataset[ifold])
print('mi-net mean accuracy = ', np.mean(acc))
print('std = ', np.std(acc))

MUSK-188     
MUSK-188     
MUSK-188     
MUSK-188     
MUSK-190     
MUSK-190     
MUSK-190     
MUSK-190     
MUSK-211     
MUSK-211     
MUSK-212     
MUSK-212     
MUSK-212     
MUSK-213     
MUSK-213     
MUSK-213     
MUSK-213     
MUSK-219     
MUSK-219     
MUSK-224     
MUSK-224     
MUSK-227     
MUSK-227     
MUSK-228     
MUSK-228     
MUSK-228     
MUSK-228     
MUSK-228     
MUSK-236     
MUSK-236     
MUSK-236     
MUSK-236     
MUSK-236     
MUSK-236     
MUSK-238     
MUSK-238     
MUSK-238     
MUSK-238     
MUSK-238     
MUSK-240     
MUSK-240     
MUSK-240     
MUSK-240     
MUSK-240     
MUSK-240     
MUSK-240     
MUSK-240     
MUSK-246     
MUSK-246     
MUSK-246     
MUSK-246     
MUSK-254     
MUSK-254     
MUSK-256     
MUSK-256     
MUSK-256     
MUSK-256     
MUSK-272     
MUSK-272     
MUSK-272     
MUSK-273     
MUSK-273     
MUSK-273     
MUSK-273     
MUSK-273     
MUSK-284     
MUSK-284     
MUSK-284     
MUSK-284     
MUSK-285     
MUSK-285     
MUSK-2

epoch= 18   train_loss= 2.081   train_acc= 1.000   test_loss=2.426   test_acc= 0.900
epoch= 19   train_loss= 2.059   train_acc= 1.000   test_loss=2.396   test_acc= 0.900
epoch= 20   train_loss= 2.065   train_acc= 0.976   test_loss=2.386   test_acc= 0.900
epoch= 21   train_loss= 2.039   train_acc= 0.976   test_loss=2.394   test_acc= 0.900
epoch= 22   train_loss= 2.029   train_acc= 1.000   test_loss=2.429   test_acc= 0.900
epoch= 23   train_loss= 2.010   train_acc= 0.976   test_loss=2.376   test_acc= 0.900
epoch= 24   train_loss= 1.978   train_acc= 1.000   test_loss=2.356   test_acc= 0.900
epoch= 25   train_loss= 1.962   train_acc= 1.000   test_loss=2.370   test_acc= 0.900
epoch= 26   train_loss= 1.946   train_acc= 1.000   test_loss=2.366   test_acc= 0.900
epoch= 27   train_loss= 1.941   train_acc= 1.000   test_loss=2.373   test_acc= 0.900
epoch= 28   train_loss= 1.922   train_acc= 1.000   test_loss=2.357   test_acc= 0.900
epoch= 29   train_loss= 1.907   train_acc= 1.000   test_loss=2.33

epoch= 14   train_loss= 2.198   train_acc= 0.976   test_loss=2.442   test_acc= 0.778
epoch= 15   train_loss= 2.180   train_acc= 0.976   test_loss=2.355   test_acc= 0.778
epoch= 16   train_loss= 2.147   train_acc= 0.964   test_loss=2.329   test_acc= 0.778
epoch= 17   train_loss= 2.141   train_acc= 0.988   test_loss=2.366   test_acc= 0.667
epoch= 18   train_loss= 2.141   train_acc= 0.964   test_loss=2.397   test_acc= 0.778
epoch= 19   train_loss= 2.089   train_acc= 0.988   test_loss=2.366   test_acc= 0.778
epoch= 20   train_loss= 2.079   train_acc= 0.976   test_loss=2.333   test_acc= 0.778
epoch= 21   train_loss= 2.063   train_acc= 0.976   test_loss=2.295   test_acc= 0.778
epoch= 22   train_loss= 2.044   train_acc= 1.000   test_loss=2.254   test_acc= 0.778
epoch= 23   train_loss= 2.030   train_acc= 0.976   test_loss=2.209   test_acc= 0.889
epoch= 24   train_loss= 2.014   train_acc= 1.000   test_loss=2.221   test_acc= 0.778
epoch= 25   train_loss= 1.978   train_acc= 1.000   test_loss=2.21

epoch= 10   train_loss= 2.322   train_acc= 0.940   test_loss=2.306   test_acc= 0.889
epoch= 11   train_loss= 2.300   train_acc= 0.952   test_loss=2.276   test_acc= 0.889
epoch= 12   train_loss= 2.265   train_acc= 0.964   test_loss=2.326   test_acc= 0.889
epoch= 13   train_loss= 2.245   train_acc= 0.976   test_loss=2.291   test_acc= 0.889
epoch= 14   train_loss= 2.179   train_acc= 0.988   test_loss=2.261   test_acc= 0.889
epoch= 15   train_loss= 2.181   train_acc= 0.952   test_loss=2.224   test_acc= 0.889
epoch= 16   train_loss= 2.165   train_acc= 0.976   test_loss=2.276   test_acc= 0.889
epoch= 17   train_loss= 2.138   train_acc= 0.976   test_loss=2.307   test_acc= 0.889
epoch= 18   train_loss= 2.099   train_acc= 0.988   test_loss=2.221   test_acc= 0.889
epoch= 19   train_loss= 2.072   train_acc= 1.000   test_loss=2.234   test_acc= 0.889
epoch= 20   train_loss= 2.062   train_acc= 1.000   test_loss=2.244   test_acc= 0.889
epoch= 21   train_loss= 2.043   train_acc= 0.988   test_loss=2.19

epoch= 6   train_loss= 2.495   train_acc= 0.867   test_loss=2.423   test_acc= 0.889
epoch= 7   train_loss= 2.424   train_acc= 0.928   test_loss=2.344   test_acc= 1.000
epoch= 8   train_loss= 2.427   train_acc= 0.880   test_loss=2.349   test_acc= 0.889
epoch= 9   train_loss= 2.384   train_acc= 0.940   test_loss=2.274   test_acc= 1.000
epoch= 10   train_loss= 2.336   train_acc= 0.916   test_loss=2.262   test_acc= 1.000
epoch= 11   train_loss= 2.274   train_acc= 0.964   test_loss=2.232   test_acc= 1.000
epoch= 12   train_loss= 2.259   train_acc= 0.940   test_loss=2.262   test_acc= 0.889
epoch= 13   train_loss= 2.225   train_acc= 0.964   test_loss=2.160   test_acc= 1.000
epoch= 14   train_loss= 2.190   train_acc= 0.964   test_loss=2.307   test_acc= 0.889
epoch= 15   train_loss= 2.192   train_acc= 0.952   test_loss=2.262   test_acc= 0.889
epoch= 16   train_loss= 2.154   train_acc= 0.988   test_loss=2.142   test_acc= 1.000
epoch= 17   train_loss= 2.122   train_acc= 0.988   test_loss=2.128   

...

epoch= 6   train_loss= 2.474   train_acc= 0.928   test_loss=2.486   test_acc= 0.778
epoch= 7   train_loss= 2.408   train_acc= 0.916   test_loss=2.483   test_acc= 0.778
epoch= 8   train_loss= 2.386   train_acc= 0.916   test_loss=2.438   test_acc= 0.778
epoch= 9   train_loss= 2.366   train_acc= 0.916   test_loss=2.483   test_acc= 0.778
epoch= 10   train_loss= 2.306   train_acc= 0.952   test_loss=2.484   test_acc= 0.778
epoch= 11   train_loss= 2.306   train_acc= 0.940   test_loss=2.488   test_acc= 0.778
epoch= 12   train_loss= 2.229   train_acc= 0.976   test_loss=2.457   test_acc= 0.778
epoch= 13   train_loss= 2.238   train_acc= 0.940   test_loss=2.411   test_acc= 0.778
epoch= 14   train_loss= 2.187   train_acc= 0.988   test_loss=2.457   test_acc= 0.778
epoch= 15   train_loss= 2.175   train_acc= 0.964   test_loss=2.387   test_acc= 0.778
epoch= 16   train_loss= 2.158   train_acc= 0.988   test_loss=2.475   test_acc= 0.778
epoch= 17   train_loss= 2.160   train_acc= 0.952   test_loss=2.373   

epoch= 2   train_loss= 2.699   train_acc= 0.855   test_loss=2.813   test_acc= 0.667
epoch= 3   train_loss= 2.655   train_acc= 0.807   test_loss=2.719   test_acc= 0.778
epoch= 4   train_loss= 2.545   train_acc= 0.880   test_loss=2.697   test_acc= 0.889
epoch= 5   train_loss= 2.500   train_acc= 0.904   test_loss=2.745   test_acc= 0.778
epoch= 6   train_loss= 2.469   train_acc= 0.892   test_loss=2.643   test_acc= 0.778
epoch= 7   train_loss= 2.425   train_acc= 0.904   test_loss=2.623   test_acc= 0.778
epoch= 8   train_loss= 2.401   train_acc= 0.904   test_loss=2.608   test_acc= 0.778
epoch= 9   train_loss= 2.341   train_acc= 0.952   test_loss=2.649   test_acc= 0.778
epoch= 10   train_loss= 2.309   train_acc= 0.940   test_loss=2.651   test_acc= 0.667
epoch= 11   train_loss= 2.284   train_acc= 0.940   test_loss=2.650   test_acc= 0.778
epoch= 12   train_loss= 2.252   train_acc= 0.952   test_loss=2.635   test_acc= 0.778
epoch= 13   train_loss= 2.202   train_acc= 0.976   test_loss=2.656   test

epoch= 48   train_loss= 1.698   train_acc= 1.000   test_loss=1.844   test_acc= 0.889
epoch= 49   train_loss= 1.693   train_acc= 1.000   test_loss=1.871   test_acc= 0.889
run time: 0.7094895680745442 min
test_acc=0.889
run= 4   fold= 6
epoch= 0   train_loss= 2.918   train_acc= 0.711   test_loss=2.843   test_acc= 0.778
epoch= 1   train_loss= 2.835   train_acc= 0.711   test_loss=2.783   test_acc= 0.667
epoch= 2   train_loss= 2.721   train_acc= 0.819   test_loss=2.692   test_acc= 0.889
epoch= 3   train_loss= 2.669   train_acc= 0.807   test_loss=2.647   test_acc= 0.778
epoch= 4   train_loss= 2.582   train_acc= 0.867   test_loss=2.607   test_acc= 0.889
epoch= 5   train_loss= 2.572   train_acc= 0.855   test_loss=2.546   test_acc= 0.889
epoch= 6   train_loss= 2.479   train_acc= 0.892   test_loss=2.504   test_acc= 0.889
epoch= 7   train_loss= 2.506   train_acc= 0.855   test_loss=2.493   test_acc= 0.889
epoch= 8   train_loss= 2.419   train_acc= 0.940   test_loss=2.471   test_acc= 0.889
epoch= 9 

epoch= 44   train_loss= 1.732   train_acc= 1.000   test_loss=3.326   test_acc= 0.778
epoch= 45   train_loss= 1.726   train_acc= 1.000   test_loss=3.319   test_acc= 0.778
epoch= 46   train_loss= 1.710   train_acc= 1.000   test_loss=3.318   test_acc= 0.778
epoch= 47   train_loss= 1.705   train_acc= 1.000   test_loss=3.291   test_acc= 0.778
epoch= 48   train_loss= 1.696   train_acc= 1.000   test_loss=3.289   test_acc= 0.778
epoch= 49   train_loss= 1.684   train_acc= 1.000   test_loss=3.270   test_acc= 0.778
run time: 0.7162322839101155 min
test_acc=0.778
run= 4   fold= 8
epoch= 0   train_loss= 3.014   train_acc= 0.627   test_loss=2.805   test_acc= 0.889
epoch= 1   train_loss= 2.784   train_acc= 0.735   test_loss=2.753   test_acc= 0.778
epoch= 2   train_loss= 2.655   train_acc= 0.855   test_loss=2.701   test_acc= 0.778
epoch= 3   train_loss= 2.597   train_acc= 0.867   test_loss=2.692   test_acc= 0.778
epoch= 4   train_loss= 2.503   train_acc= 0.880   test_loss=2.694   test_acc= 0.778
epoch

epoch= 40   train_loss= 1.807   train_acc= 1.000   test_loss=1.882   test_acc= 1.000
epoch= 41   train_loss= 1.796   train_acc= 1.000   test_loss=1.873   test_acc= 1.000
epoch= 42   train_loss= 1.787   train_acc= 1.000   test_loss=1.834   test_acc= 1.000
epoch= 43   train_loss= 1.777   train_acc= 1.000   test_loss=1.833   test_acc= 1.000
epoch= 44   train_loss= 1.771   train_acc= 1.000   test_loss=1.803   test_acc= 1.000
epoch= 45   train_loss= 1.756   train_acc= 1.000   test_loss=1.808   test_acc= 1.000
epoch= 46   train_loss= 1.751   train_acc= 1.000   test_loss=1.796   test_acc= 1.000
epoch= 47   train_loss= 1.734   train_acc= 1.000   test_loss=1.791   test_acc= 1.000
epoch= 48   train_loss= 1.733   train_acc= 1.000   test_loss=1.797   test_acc= 1.000
epoch= 49   train_loss= 1.719   train_acc= 1.000   test_loss=1.780   test_acc= 1.000
run time: 0.7182445844014486 min
test_acc=1.000
mi-net mean accuracy =  0.8300000059604645
std =  0.10402396848155698
